In [560]:
import re
import os
import spacy
import pyreadr
import warnings
import openpyxl
import numpy as np
import pandas as pd
from datetime import datetime
from autocorrect import Speller
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

warnings.filterwarnings('ignore')

nlp = spacy.load("es_core_news_sm")
spell = Speller(lang='es')

# !python -m spacy download es_core_news_sm
# stop_words = spacy.lang.es.stop_words.STOP_WORDS

# C2 Sur, C2 Norte, C2 Centro, C2 Oriente, C2 Poniente, C2 Centro Histórico
C2 = 'C2 Centro Histórico'


In [561]:
negatives=['(NO|NADIE|NEGATIVO|SIN|AUSENCIA)\s*([SEÉéLQUDHYBRAlI]*\s)*\s*[LOGRACEPNXIM]*\s*[GUNLQIEOA]*\s*[ÉEL]*\s*ENTREVISTA[RSE]*|(NEGATIVO DE QUE ALGUIEN SALGA A ENTREVISTARSE)|(ausencia\s*(de)?\s*alguno\s*(persona)?\s*(con)?\s*(el)?\s*(que)?\s*(él)?\s*entrevista)']

reasons = ['A\s*M\s*P\s*L\s*I\s*O\s*E\s*L\s*R\s*E\s*C\s*O\s*R\s*R\s*I\s*D\s*O',
           'AMPLI[OARN]+\s*(NUEVAMENTE)?\s*[DNELA\s*]*\s*(RECORRIDO|BUSQUEDA|PERIMETRO|INSPECCION|REVISON|LUGAR|INVESTIGACION|REVISION|PARA DESCARTAR)']

asi = 'AFIRMATIVO SIN INTERVENCIÓN'
aci = 'AFIRMATIVO CON INTERVENCIÓN'

monts = {'01': 'Enero', '02': 'Febrero', '03': 'Marzo', '04': 'Abril', '05': 'Mayo', '06': 'Junio', '07': 'Julio', '08': 'Agosto', '09': 'Septiembre', '10': 'Octubre', '11': 'Noviembre', '12': 'Diciembre'}

negatives = [re.compile(negative, re.IGNORECASE) for negative in negatives]
reasons = [re.compile(reson, re.IGNORECASE) for reson in reasons]

In [562]:
def lematizador(textos):
    i=0
    comentarios_lema = []
    for comentario in textos:
        i+=1
        print(i, "/", len(textos), end='\r')
        doc = nlp(comentario)
        comentario_aux = ''
        for token in doc:
            comentario_aux += token.lemma_ + ' '
        comentarios_lema.append(comentario_aux)
    return comentarios_lema

# Diccionario

In [563]:
# si diccionario_lematizado.xlsx no existe, se crea
try:
    diccionario = pd.read_excel('diccionario.xlsx', dtype=str)
    diccionario.head()
except:
    diccionario = pd.DataFrame(columns=['palabra', 'corrección'])
    diccionario.to_excel('diccionario.xlsx', index=False)
diccionario.head()

,palabra,corrección
0,colro,color
1,califa,califa
2,lugo,lugo
3,siin,sin
4,ponciano,anciano


In [564]:
# 1 Nombre de la base
df = pd.read_csv('folios_nadie_solicita_apoyo_ 2023-02-01 _al_ 2023-04-16 .csv', sep=',')

delegacionesC2 ={'C2 Sur' : 'BENITO JUAREZ, COYOACAN, TLALPAN',
            'C2 Norte' : 'IZTACALCO, GUSTAVO A. MADERO, VENUSTIANO CARRANZA',
            'C2 Centro' : 'CUAUHTEMOC',
            'C2 Oriente' : 'IZTAPALAPA, XOCHIMILCO, MILPA ALTA, TLAHUAC',
            'C2 Poniente' : 'ALVARO OBREGON, AZCAPOTZALCO, MAGDALENA CONTRERAS, MIGUEL HIDALGO, CUAJIMALPA',
            }

# sector 
# 2 Nombre del C2

if not os.path.exists(C2):
    os.makedirs(C2)

# 3 Temporalidad
now = datetime.now()
day = now.weekday()
if day == 6:
    fecha_fin = str(now.strftime('%Y-%m-%d'))
else:
    fecha_fin = str(now.replace(day=now.day-int(day)-1).strftime('%Y-%m-%d'))
fecha_inicio = str(now.replace(day=1, month=now.month-1).strftime('%Y-%m-%d'))

if C2 == 'C2 Centro' or C2 == 'C2 Norte' or C2 == 'C2 Centro Histórico':
    result = pyreadr.read_r('ECHO_V2.RDS')
    data = next(iter(result.values()))
    data = data[data['c2_final'] == 'C2 Centro Histórico']
    data = data.reset_index(drop=True)
    if C2 != 'C2 Centro Histórico':
        # eliminar de df los registros que tengan el mismo folio de df y data y tengan en la columna c2_final C2 Centro Histórico
        df = df[~df['folio'].isin(data['folio'])]
        df = df.reset_index(drop=True)
if C2 == 'C2 Centro Histórico':
    df = df[df['folio'].isin(data['folio'])]
else:
    delegaciones = delegacionesC2[C2]
    delegaciones = delegaciones.split(', ')
    df = df[df['delegacion_cierre'].isin(delegaciones)]

df['Mes'] = df['fecha_cierre'].astype(str).str.extract(r'-(\d\d)', expand=False)
df['Mes'] = df['Mes'].map(monts)
df = df[(df['fecha_cierre'] >= fecha_inicio) & (df['fecha_cierre'] <= fecha_fin)]

df_copy = df.copy()

A = len(df[df['codigo_cierre'] == 'A'])
D = len(df[df['codigo_cierre'] == 'D'])
I = len(df[df['codigo_cierre'] == 'I'])
F = len(df[df['codigo_cierre'] == 'F'])

df = df[df['codigo_cierre'] == 'A']
df = df.reset_index(drop=True)
df_copy = df_copy.reset_index(drop=True)
print("A: ",A, "\tD: ",D, "\tI: ",I, "\tF: ",F)

A:  24 	D:  0 	I:  0 	F:  76


In [565]:
df = df.replace('Ã\x81', 'Á', regex=True)
df = df.replace('Ã¡', 'á', regex=True)
df = df.replace('Ã‰', 'É', regex=True)
df = df.replace('Ã©', 'é', regex=True)
df = df.replace('Ã\x8d', 'Í', regex=True)
df = df.replace('Ã\xad', 'í', regex=True)
df = df.replace('Ã“', 'Ó', regex=True)
df = df.replace('Ã³', 'ó', regex=True)
df = df.replace('Ãš', 'Ú', regex=True)
df = df.replace('Ãº', 'ú', regex=True)
df = df.replace('Ã‘', 'Ñ', regex=True)
df = df.replace('Ã±', 'ñ', regex=True)
df = df.replace('Â°', '°', regex=True)
df = df.reset_index(drop=True)
# df = df[df['comentarios.y'].str.contains('NADIE SOLICITA EL APOYO', na=False)]

# TABLA NSA

In [566]:
df_tabla = df[['folio']]
df_tabla['CR'] = df[['codigo_cierre']]
df_tabla['CI'] = 0
df_tabla['CG'] = 0
df_tabla['Mes'] = df['Mes']
df_tabla['Clasificación'] = df['clas_con_f_alarma']
df_tabla.loc[df_tabla['Clasificación'].str.contains('DELITO', na=False), 'Clasificación'] = 'DELITOS'
df_tabla.loc[df_tabla['Clasificación'] != 'DELITOS', 'Clasificación'] = 'OTROS'
df_tabla['Amplio recorrido'] = np.nan
df_tabla['Entrevista'] = np.nan
df_tabla['Incidente'] = df[['incidente_c4']]

df_tabla['CG'] = df['comentarios.y'].str.extract(r'CIERRE GLOBAL: (.+?);', expand=False)

df_tabla.loc[df_tabla['CG'].str.contains(asi, na=False), 'CI'] = 'ASI'
df_tabla.loc[df_tabla['CG'].str.contains(aci, na=False), 'CI'] = 'ACI'
df_tabla.loc[df_tabla['CI'] == 0, 'CI'] = np.nan

df_tabla

,folio,CR,CI,CG,Mes,Clasificación,Amplio recorrido,Entrevista,Incidente
0,C2C/20230315/00143,A,ASI,AFIRMATIVO SIN INTERVENCIÓN,Marzo,DELITOS,NaN,NaN,Agresión-Violencia contra la Mujer
1,C2C/20230327/00168,A,ASI,AFIRMATIVO SIN INTERVENCIÓN,Marzo,DELITOS,NaN,NaN,Agresión-Violencia contra la Mujer
2,C5/20230303/02086,A,ACI,AFIRMATIVO CON INTERVENCIÓN,Marzo,DELITOS,NaN,NaN,Agresión-Persona
3,C5/20230303/03533,A,ASI,AFIRMATIVO SIN INTERVENCIÓN,Marzo,DELITOS,NaN,NaN,Agresión-Violencia contra la Mujer
4,C5/20230303/05125,A,ASI,AFIRMATIVO SIN INTERVENCIÓN,Marzo,DELITOS,NaN,NaN,Agresión-Violencia contra la Mujer
5,C5/20230304/05223,A,ASI,AFIRMATIVO SIN INTERVENCIÓN,Marzo,DELITOS,NaN,NaN,Agresión-Violencia contra la Mujer
6,C5/20230309/03752,A,ASI,AFIRMATIVO SIN INTERVENCIÓN,Marzo,DELITOS,NaN,NaN,Sexuales-Acoso
7,C5/20230310/00696,A,ASI,AFIRMATIVO SIN INTERVENCIÓN,Marzo,DELITOS,NaN,NaN,Agresión-Violencia contra la Mujer
8,C5/20230314/03370,A,ACI,AFIRMATIVO CON INTERVENCIÓN,Marzo,OTROS,NaN,NaN,Accidente-Choque sin Lesionados
9,C5/20230318/04031,A,ASI,AFIRMATIVO SIN INTERVENCIÓN,Marzo,DELITOS,NaN,NaN,Agresión-Persona


In [567]:
elementos = ['SE AGREGÓ UNA DESCRIPCIÓN: ', 'SE AGREGÓ NOTA DE CIERRE: ', 'SE AGREGÓ CIERRE GLOBAL: ',]
for i in range(len(df)):
    # df['comentarios.y'][i] = df['comentarios.y'][i][df['comentarios.y'][i].find('NADIE SOLICITA EL APOYO'):]
    df['comentarios.y'][i] = re.split('\[\d{2}\/\d{2}\/\d{4}\s\d{2}:\d{2}:\d{2}\]\s', df['comentarios.y'][i])
    df_aux = []
    for j in range(len(df['comentarios.y'][i])):
       if any(elemento in df['comentarios.y'][i][j] for elemento in elementos):
           df_aux.append(df['comentarios.y'][i][j])
    df['comentarios.y'][i] = df_aux
    df['comentarios.y'][i] = [x for x in df['comentarios.y'][i] if any(elemento in x for elemento in elementos)]

In [568]:
for comentario in range(len(df)):
    for j in range(len(df['comentarios.y'][comentario])):
        df['comentarios.y'][comentario][j] = re.sub('\[\w+\](\s*[\wÀ-Ý]+\s*)+:\s*', '', df['comentarios.y'][comentario][j])
    df['comentarios.y'][comentario] = ' '.join(df['comentarios.y'][comentario])
    df['comentarios.y'][comentario] = re.sub('[^a-zA-ZñÑáéíóúÁÉÍÓÚ\s]', '', df['comentarios.y'][comentario])
    df['comentarios.y'][comentario] = re.sub('\s+', ' ', df['comentarios.y'][comentario])
df['comentarios.y'] = df['comentarios.y'].str.lower()

df['comentarios.y'] = lematizador(df['comentarios.y'])

for i in range(len(df)):
    df['comentarios.y'][i] = re.sub('nombre\w+', 'nombre', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('\w+tipo', 'tipo', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('submarca\w+', 'submarca', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('marca\w+', 'marca', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('placa\w+', 'placa', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('\w+masculino', 'masculino', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('\w+femenino', 'femenino', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('color\w+', 'color', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('\w+color', 'color', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('colonia\w+', 'colomia', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('particulares\w+', 'particulares', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('http[\w./\-#;:%_&$!?¡¿"()[]]+', '', df['comentarios.y'][i])
    df['comentarios.y'][i] = re.sub('\w+informar', 'informar', df['comentarios.y'][i])

In [569]:
palabras_no_encontradas = []
for i in range(len(df)):
    print(i+1, "/", len(df), end="\r")
    for palabra in df['comentarios.y'][i].split():
        # si la palabra está en el diccionario, reemplazarla por la corrección solo si la plabra esta entre espacios
        if palabra in diccionario['palabra'].values:
            df['comentarios.y'][i] = re.sub(r'\b' + palabra + r'\b', str(diccionario['corrección'][diccionario['palabra'] == palabra].values[0]), df['comentarios.y'][i])
        else:
            if palabra not in palabras_no_encontradas:
                palabras_no_encontradas.append(palabra)
                aux = palabra
                palabra = spell(palabra)
                if palabra in diccionario['palabra'].values:
                    df['comentarios.y'][i] = re.sub(r'\b' + palabra + r'\b', str(diccionario['corrección'][diccionario['palabra'] == palabra].values[0]), df['comentarios.y'][i])
                else:
                    doc = nlp(palabra)
                    for token in doc:
                        palabra = token.lemma_
                    if palabra in diccionario['palabra'].values:
                        df['comentarios.y'][i] = re.sub(r'\b' + palabra + r'\b', str(diccionario['corrección'][diccionario['palabra'] == palabra].values[0]), df['comentarios.y'][i])
                        palabras_no_encontradas.remove(aux)

In [570]:
# palabras_no_encontradas

In [571]:
df['Amplio recorrido'] = np.nan
df['Entrevista'] = np.nan
df['Categoria'] = np.nan

df_copy['Amplio recorrido'] = np.nan
df_copy['Entrevista'] = np.nan
df_copy['Categoria'] = np.nan

In [572]:
pol = 'recorrido pa'
# expresion regular para buscar la palabra
re.search(r'\b' + pol + r'\b', df['comentarios.y'][0])


In [573]:
matches = []
matches2 = []
matches3 = []

for i in range(len(df)):
    for reason in reasons:
        if reason.search(df['comentarios.y'][i]):
            # print(i)
            # matches.append(df_copy['comentarios.y'][i])
            matches.append(reason.search(df_copy['comentarios.y'][i]))
            df_tabla['Amplio recorrido'][i] = 1
            df['Amplio recorrido'][i] = 1
            break

for i in range(len(df)):
    for negative in negatives:
        if negative.search(df['comentarios.y'][i]):
            df_tabla['Entrevista'][i] = 0
            # matches2.append(df_copy['comentarios.y'][i])
            matches.append(negative.search(df_copy['comentarios.y'][i]))
            break
    if 'entrevista' in df['comentarios.y'][i] and df_tabla['Entrevista'][i] != 0:
        # print(i)
        matches3.append(df['comentarios.y'][i][df['comentarios.y'][i].find('entrevista')-40:df['comentarios.y'][i].find('entrevista')+40])
        df_tabla['Entrevista'][i] = 1
        df['Entrevista'][i] = 1

# fill nan values with 0
df_tabla['Entrevista'] = df_tabla['Entrevista'].fillna(0)
df_tabla['Amplio recorrido'] = df_tabla['Amplio recorrido'].fillna(0)


In [574]:
# matches

In [575]:
df['Amplio recorrido'] = df['Amplio recorrido'].fillna(0)
df['Entrevista'] = df['Entrevista'].fillna(0)

# categorias RE, RNE, NRE, NRNE
for i in range(len(df)):
    if df['Amplio recorrido'][i] == 1 and df['Entrevista'][i] == 1:
        df['Categoria'][i] = 'RE'
    elif df['Amplio recorrido'][i] == 1 and df['Entrevista'][i] == 0:
        df['Categoria'][i] = 'RNE'
    elif df['Amplio recorrido'][i] == 0 and df['Entrevista'][i] == 1:
        df['Categoria'][i] = 'NRE'
    elif df['Amplio recorrido'][i] == 0 and df['Entrevista'][i] == 0:
        df['Categoria'][i] = 'NRNE'

In [576]:
for i in range(len(df)):
    for j in range(len(df_copy)):
        if df['folio'][i] == df_copy['folio'][j]:
            df_copy['Amplio recorrido'][j] = df['Amplio recorrido'][i]
            df_copy['Entrevista'][j] = df['Entrevista'][i]
            df_copy['Categoria'][j] = df['Categoria'][i]

In [577]:
# cuantos incidentes tienen CR "A"
print("Afirmativos: ",df_tabla[df_tabla['CR'] == 'A'].shape[0])
print("Amplio recorrido: ",df_tabla[(df_tabla['Amplio recorrido'] == 1) & (df_tabla['CR'] == 'A')].shape[0])
print("Entrevista: ",df_tabla[(df_tabla['Entrevista'] == 1) & (df_tabla['CR'] == 'A')].shape[0])

# cuantos tienen entrevista y amplio recorrido y CR "A"
print("Entrevista y amplio recorrido: ",RE:=df_tabla[(df_tabla['Entrevista'] == 1) & (df_tabla['Amplio recorrido'] == 1) & (df_tabla['CR'] == 'A')].shape[0])
# cuantos tienen 0 en entrevista y 1 en amplio recorrido y CR "A"
print("No entrevista y amplio recorrido: ",RNE:=df_tabla[(df_tabla['Entrevista'] == 0) & (df_tabla['Amplio recorrido'] == 1) & (df_tabla['CR'] == 'A')].shape[0])
# cuantos tienen 1 en entrevista y 0 en amplio recorrido y CR "A"
print("Entrevista y no amplio recorrido: ",NRE:=df_tabla[(df_tabla['Entrevista'] == 1) & (df_tabla['Amplio recorrido'] == 0) & (df_tabla['CR'] == 'A')].shape[0])
# cuantos tienen 0 en entrevista y 0 en amplio recorrido y CR "A"
print("No entrevista y no amplio recorrido: ",NRNE:=df_tabla[(df_tabla['Entrevista'] == 0) & (df_tabla['Amplio recorrido'] == 0) & (df_tabla['CR'] == 'A')].shape[0])

Afirmativos:  24
Amplio recorrido:  5
Entrevista:  6
Entrevista y amplio recorrido:  2
No entrevista y amplio recorrido:  3
Entrevista y no amplio recorrido:  4
No entrevista y no amplio recorrido:  15


In [578]:
wb = openpyxl.load_workbook('Tabla y graficas.xlsx')
sheet = wb.get_sheet_by_name('Tabla y graficas')

meses = df_tabla['Mes'].unique()

sheet['A2'] = meses[0]
sheet['A4'] = meses[1]

sheet['C2'] = len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['clas_con_f_alarma2']  == 'DELITOS')])
sheet['C3'] = len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['clas_con_f_alarma2']  == 'OTROS')])
sheet['C4'] = len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['clas_con_f_alarma2']  == 'DELITOS')])
sheet['C5'] = len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['clas_con_f_alarma2']  == 'OTROS')])
sheet['C6'] = len(df_copy)

sheet['D2'] = len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['codigo_cierre'] == 'A') & (df_copy['clas_con_f_alarma2']  == 'DELITOS')])
sheet['D3'] = len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['codigo_cierre'] == 'A') & (df_copy['clas_con_f_alarma2']  == 'OTROS')])
sheet['D4'] = len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['codigo_cierre'] == 'A') & (df_copy['clas_con_f_alarma2']  == 'DELITOS')])
sheet['D5'] = len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['codigo_cierre'] == 'A') & (df_copy['clas_con_f_alarma2']  == 'OTROS')])
sheet['D6'] = '=SUM(D2:D5)'

sheet['E2'] = len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['Entrevista'] == 1) & (df_copy['clas_con_f_alarma2'] == 'DELITOS')])
sheet['E3'] = len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['Entrevista'] == 1) & (df_copy['clas_con_f_alarma2'] == 'OTROS')])
sheet['E4'] = len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['Entrevista'] == 1) & (df_copy['clas_con_f_alarma2'] == 'DELITOS')])
sheet['E5'] = len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['Entrevista'] == 1) & (df_copy['clas_con_f_alarma2'] == 'OTROS')])
sheet['E6'] = '=SUM(E2:E5)'

sheet['F2'] = sheet['D2'].value-sheet['E2'].value
sheet['F3'] = sheet['D3'].value-sheet['E3'].value
sheet['F4'] = sheet['D4'].value-sheet['E4'].value
sheet['F5'] = sheet['D5'].value-sheet['E5'].value
sheet['F6'] = sheet['F2'].value+sheet['F3'].value+sheet['F4'].value+sheet['F5'].value

def porcentaje(valor, total):
    if total == 0:
        return 0
    else:
        return valor/total

sheet['G2'] = porcentaje(sheet['F2'].value, len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['clas_con_f_alarma2']  == 'DELITOS')]))
sheet['G3'] = porcentaje(sheet['F3'].value, len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['clas_con_f_alarma2']  == 'OTROS')]))
sheet['G4'] = porcentaje(sheet['F4'].value, len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['clas_con_f_alarma2']  == 'DELITOS')]))
sheet['G5'] = porcentaje(sheet['F5'].value, len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['clas_con_f_alarma2']  == 'OTROS')]))
sheet['G6'] = sheet['F6'].value/len(df_copy)

if sheet['G4'].value > sheet['G2'].value:
    sheet['G4'].fill = PatternFill(start_color='ff0000', end_color='ff0000', fill_type='solid')
else:
    sheet['G4'].fill = PatternFill(start_color='92d050', end_color='92d050', fill_type='solid')

if sheet['G5'].value > sheet['G3'].value:
    sheet['G5'].fill = PatternFill(start_color='ff0000', end_color='ff0000', fill_type='solid')
else:
    sheet['G5'].fill = PatternFill(start_color='92d050', end_color='92d050', fill_type='solid')

sheet['G11'] = len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['Amplio recorrido'] == 0)])/(len(df_copy[(df_copy['Mes'] == meses[0])]))
sheet['G12'] = len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['Amplio recorrido'] == 0)])/(len(df_copy[(df_copy['Mes'] == meses[1])]))

sheet['G15'] = len(df_copy[(df_copy['Mes'] == meses[0]) & (df_copy['Entrevista'] == 1)])/(len(df_copy[(df_copy['Mes'] == meses[0])]))
sheet['G16'] = len(df_copy[(df_copy['Mes'] == meses[1]) & (df_copy['Entrevista'] == 1)])/(len(df_copy[(df_copy['Mes'] == meses[1])]))

sheet['G28'] = A
sheet['G29'] = D
sheet['G30'] = F
sheet['G31'] = I

sheet['G34'] = len(df_copy[(df_copy['Amplio recorrido'] == 1)])
sheet['G35'] = len(df_copy[(df_copy['Amplio recorrido'] == 0)])

sheet['G38'] = RE
sheet['G39'] = RNE
sheet['G40'] = NRE
sheet['G41'] = NRNE

wb.save(f'{C2}/NSA_{C2}_{fecha_inicio}_{fecha_fin}.xlsx')

In [579]:

wb = load_workbook(f'{C2}/NSA_{C2}_{fecha_inicio}_{fecha_fin}.xlsx')

wb.create_sheet('Tabla')
sheet = wb['Tabla']

for r in dataframe_to_rows(df_tabla, index=False, header=True):
    sheet.append(r)

wb.create_sheet('NSA')
sheet2 = wb['NSA']

for r in dataframe_to_rows(df_copy, index=False, header=True):
    sheet2.append(r)

wb.save(f'{C2}/NSA_{C2}_{fecha_inicio}_{fecha_fin}.xlsx')


In [580]:
# import plotly.graph_objects as go
# import pandas as pd

# df['Categoria'].value_counts()

# data = {'Categoria': ['N_folios'],
#         'Cantidad_RE': [df['Categoria'].value_counts()['RE']],
#         'Cantidad_RNE': [df['Categoria'].value_counts()['RNE']],
#         'Cantidad_NRE': [df['Categoria'].value_counts()['NRE']],
#         'Cantidad_NRNE': [df['Categoria'].value_counts()['NRNE']]}

# df_grafica = pd.DataFrame(data)

# fig = go.Figure(data=[
#         go.Bar(name='RE', x=df_grafica['Categoria'], y=df_grafica['Cantidad_RE'], marker_color='#bfbfbf', text=df_grafica['Cantidad_RE'], textfont=dict(size=1800), outsidetextfont=dict(size=1800)),
#         go.Bar(name='RNE', x=df_grafica['Categoria'], y=df_grafica['Cantidad_RNE'], marker_color='#c00000', text=df_grafica['Cantidad_RNE']),       
#         go.Bar(name='NRE', x=df_grafica['Categoria'], y=df_grafica['Cantidad_NRE'], marker_color='#bfbfbf', text=df_grafica['Cantidad_NRE']),
#         go.Bar(name='NRNE', x=df_grafica['Categoria'], y=df_grafica['Cantidad_NRNE'], marker_color='#c00000', text=df_grafica['Cantidad_NRNE'])
#         ])

# fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
# fig.update_layout(width=300, height=500)
# fig.show()

